# HYPERPARAMETER TUNING

In [1]:
import logging
import byron

/home/saccuz/Scrivania/TESI/byron/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_30192/326451194.py:2: ByronPerformanceWarning: 
  Paranoia checks are enabled in this notebook: performances can be significantly impaired
  [see https://cad-polito-it.github.io/byron/paranoia for details]
  import byron


In [2]:
byron.logger.setLevel(level=logging.CRITICAL)


In [3]:
NUM_BITS = 50


In [4]:
@byron.fitness_function
def fitness(genotype):
    """Parametric 1-max"""
    return sum(b == '1' for b in genotype)

In [5]:
macro = byron.f.macro('{v}', v=byron.f.array_parameter('01', NUM_BITS + 1))
top_frame = byron.f.sequence([macro])

In [6]:
evaluator = byron.evaluator.PythonEvaluator(fitness, strip_phenotypes=True)

## Default configuration

In [7]:
max_generation=500
lambda_=20
mu=30
top_n=5
lifespan=100
rewards=[0.7,0.3]
temperature = 0.85

In [8]:
# population = byron.ea.adaptive_ea(
#     top_frame, evaluator, max_generation=max_generation, lambda_=lambda_, mu=mu, max_fitness=NUM_BITS, top_n=top_n, lifespan=lifespan, rewards=rewards, temperature=temperature
# )

## Tuning mu, lambda


In [9]:
output_file = open('mu_lambda_mean.txt', 'a')

In [10]:
from math import ceil
from byron.sys import get_operators
from time import time

In [11]:
def log_operators(f):
    all_ops = sorted(get_operators(), key=lambda o: (o.num_parents if o.num_parents is not None else -1, o.__name__))

    descr = {None: 'init', 1: 'mut', 2: 'xover'}

    f.write(f"GENETIC OPERATORS\n")
    for op in all_ops:
        name = f'{op.__name__} ({descr[op.num_parents]})'
        f.write(f"* {name:.<50s}: {op.stats}\n")

In [12]:
#logger.addHandler(logging.StreamHandler(output_file))

In [13]:
for mu in range(10, 160, 10):
    lambda_ = mu - ceil(mu*0.1)
    mean_time = []

    #output_file.write(f"MU VALUE: {mu} LAMBDA VALUE: {lambda_}\n")
    for _ in range(5):
        t1 = time()

        population = byron.ea.adaptive_ea(
            top_frame, evaluator, max_generation=max_generation, lambda_=lambda_, mu=mu, max_fitness=NUM_BITS, top_n=top_n, lifespan=lifespan, rewards=rewards, temperature=temperature
        )

        t2 = time()

        #output_file.write(f"\nTime: {t2-t1}\n")
        #output_file.write(f"AdaptiveEA: 🍀 {population[0].describe(include_fitness=True, include_structure=False, include_age=True, include_lineage=False)}"
        #    + f" [🕓 gen: {population.generation:,} / fcalls: {evaluator.fitness_calls:,}]")
        #log_operators(output_file)
        mean_time.append(t2-t1)
    output_file.write(f"time: {sum(mean_time)/len(mean_time)}\n")
output_file.write("\n\n END LESS\n\n")

/home/saccuz/Scrivania/TESI/byron/byron/ea/adaptive.py:150: ByronPerformanceWarning: 
  The failure rate of 'node_crossover_unfussy' is 100% (0 successes out of 100 calls)
  new_individuals += op(*parents)
/home/saccuz/Scrivania/TESI/byron/byron/ea/adaptive.py:148: ByronPerformanceWarning: 
  The failure rate of 'add_macro_to_bunch' is 100% (0 successes out of 100 calls)
  new_individuals += op(*parents, strength=sigma)
/home/saccuz/Scrivania/TESI/byron/byron/ea/adaptive.py:148: ByronPerformanceWarning: 
  The failure rate of 'remove_macro_from_bunch' is 100% (0 successes out of 100 calls)
  new_individuals += op(*parents, strength=sigma)
/home/saccuz/Scrivania/TESI/byron/byron/ea/adaptive.py:150: ByronPerformanceWarning: 
  The failure rate of 'node_crossover_choosy' is 100% (0 successes out of 100 calls)
  new_individuals += op(*parents)


13

In [14]:
for mu in range(10, 160, 10):
    lambda_ = mu + ceil(mu*0.1)
    mean_time = []

    #output_file.write(f"MU VALUE: {mu} LAMBDA VALUE: {lambda_}\n")
    for _ in range(5):
        t1 = time()

        population = byron.ea.adaptive_ea(
            top_frame, evaluator, max_generation=max_generation, lambda_=lambda_, mu=mu, max_fitness=NUM_BITS, top_n=top_n, lifespan=lifespan, rewards=rewards, temperature=temperature
        )

        t2 = time()

        #output_file.write(f"\nTime: {t2-t1}\n")
        #output_file.write(f"AdaptiveEA: 🍀 {population[0].describe(include_fitness=True, include_structure=False, include_age=True, include_lineage=False)}"
        #    + f" [🕓 gen: {population.generation:,} / fcalls: {evaluator.fitness_calls:,}]")
        #log_operators(output_file)
        mean_time.append(t2-t1)
    output_file.write(f"time: {sum(mean_time)/len(mean_time)}\n")
output_file.write("\n\n END MORE\n\n")

/home/saccuz/Scrivania/TESI/byron/byron/ea/adaptive.py:150: ByronPerformanceWarning: 
  The failure rate of 'node_crossover_choosy' is 100% (0 successes out of 1,000 calls)
  new_individuals += op(*parents)
/home/saccuz/Scrivania/TESI/byron/byron/ea/adaptive.py:150: ByronPerformanceWarning: 
  The failure rate of 'node_crossover_unfussy' is 100% (0 successes out of 1,000 calls)
  new_individuals += op(*parents)
/home/saccuz/Scrivania/TESI/byron/byron/ea/adaptive.py:148: ByronPerformanceWarning: 
  The failure rate of 'remove_macro_from_bunch' is 100% (0 successes out of 1,000 calls)
  new_individuals += op(*parents, strength=sigma)
/home/saccuz/Scrivania/TESI/byron/byron/ea/adaptive.py:148: ByronPerformanceWarning: 
  The failure rate of 'add_macro_to_bunch' is 100% (0 successes out of 1,000 calls)
  new_individuals += op(*parents, strength=sigma)


13

In [15]:
# mu = 10
# lambda_ = 20
# output_file.write(f"\n")
# output_file.write(f"MU VALUE: {mu} LAMBDA VALUE: {lambda_}\n")

# t1 = time()

# population = byron.ea.adaptive_ea(
#     top_frame, evaluator, max_generation=max_generation, lambda_=lambda_, mu=mu, max_fitness=NUM_BITS, top_n=top_n, lifespan=lifespan, rewards=rewards, temperature=temperature
# )

# t2 = time()

# output_file.write(f"\nTime: {t2-t1}\n")
# output_file.write(f"AdaptiveEA: 🍀 {population[0].describe(include_fitness=True, include_structure=False, include_age=True, include_lineage=False)}"
#     + f" [🕓 gen: {population.generation:,} / fcalls: {evaluator.fitness_calls:,}]")
# log_operators(output_file)

In [16]:
output_file.close()

In [17]:
# mu = 10
# lambda_ = 11
# print(f"\n")
# print(f"MU VALUE: {mu} LAMBDA VALUE: {lambda_}\n")
# mean_time = []
# for _ in range(5):
#     t1 = time()

#     population = byron.ea.adaptive_ea(
#         top_frame, evaluator, max_generation=max_generation, lambda_=lambda_, mu=mu, max_fitness=NUM_BITS, top_n=top_n, lifespan=lifespan, rewards=rewards, temperature=temperature
#     )

#     t2 = time()

#     print(f"\nTime: {t2-t1}\n")
#     print(f"AdaptiveEA: 🍀 {population[0].describe(include_fitness=True, include_structure=False, include_age=True, include_lineage=False)}"
#         + f" [🕓 gen: {population.generation:,} / fcalls: {evaluator.fitness_calls:,}]")
#     mean_time.append(t2-t1)
# print(sum(mean_time)/len(mean_time))
